In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import json
from time import sleep, perf_counter as pf
import itertools as it

In [ ]:
date = pd.DataFrame(pd.date_range('01/01/2015', '31/12/2017', freq='D'), columns=['Date'])
date.insert(0, 'Date_key', date.index)
display(date)

In [ ]:
# Add date information
date['Day_of_week'] = date.Date.dt.dayofweek
date['Year'] = date.Date.dt.year
date['Month'] = date.Date.dt.month
date['Day'] = date.Date.dt.day
date['Week'] = date.Date.dt.week
date['Weekend'] = 0
date.loc[
    (date.Day_of_week == 0) |
    (date.Day_of_week == 6),
    'Weekend'
] = 1
date['season'] = (date.Date.dt.month%12 + 3)//3
date.to_csv('final/date.csv', index=False)
display(date)

In [ ]:
# Add Crime
crime = pd.read_csv('datasets/crime_output.csv')
date.Date.astype(str)
crime['DATE'] = pd.to_datetime(crime['DATE'])
crime['start_dt'] = pd.to_datetime(crime['start_dt'])
crime['end_dt'] = pd.to_datetime(crime['end_dt'])
df = pd.merge(date, crime, left_on='Date', right_on='DATE', how='left')
display(df)

In [ ]:
# Add Location
location = pd.read_csv('datasets/final/location.csv')
df['NEIGHBORHOOD'] = df.NEIGHBORHOOD.str.strip()
df['NEIGHBORHOOD'] = df.NEIGHBORHOOD.str.lower()
df['NEIGHBORHOOD'] = df.NEIGHBORHOOD.str.replace('-', ' ')
df['NEIGHBORHOOD'] = df.NEIGHBORHOOD.str.replace('   ', ' ')

df = pd.merge(df, location, left_on=['ADDRESS', 'LAT', 'LON', 'NEIGHBORHOOD'], right_on=['ADDRESS', 'LAT', 'LON', 'NEIGHBORHOOD'], how='left')
display(df)

In [ ]:
# Add Weather
weather = pd.read_csv('datasets/weather.csv')
weather['Date'] = pd.to_datetime(weather['Date'])
df = pd.merge(df, weather, left_on=['City', 'Date'], right_on=['City', 'Date'], how='left')
display(df)

In [ ]:
# Add is_nighttime
df['Is_Nighttime'] = 1
df.loc[(df.TIME > df.Sunrise) & (df.TIME < df.Sunset), 'Is_Nighttime'] = 0
display(df)

In [ ]:
# Add stock data
stock = pd.read_csv('datasets/StockData.csv')
stock['date'] = pd.to_datetime(stock.date)
df = pd.merge(df, stock, left_on='Date', right_on='date', how='left')
display(df)

In [ ]:
# Add Events
events = pd.read_csv('datasets/events_mapping.csv')
events['date'] = pd.to_datetime(events.date)
df = pd.merge(df, events, left_on='Date', right_on='date', how='left')
display(df)

In [ ]:
final = df[['Date_key', 'eventKey', 'Crime_key', 'Location_key', 'Weather_key', 'stockKey', 'IS_FATAL', 'IS_TRAFFIC', 'Is_Nighttime']]
display(final)
final.to_csv('final/fact.csv', index=False)